In [1]:
import sys
import os
import numpy as np
import pyvista as pv
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
from helpers import safe_parse_quantity, delete_pyvista_fields, map_on_control_mesh
import pint
import vtk
from tqdm import tqdm

In [2]:
# Add it to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.helper import calculate_normal

In [3]:
my_cmap = plt.get_cmap("coolwarm", 15)
root = Path().cwd().parent / "Snapshots"
print(root)
print(root.exists())

/Users/thomassimader/Documents/NIRB/Snapshots
True


In [ ]:
PARAMETER_SPACE = "01"
DATA_TYPE = "Test"

In [ ]:
path_domain_3d = root / PARAMETER_SPACE / "Training" / "Training_001.vtu"
domain_3d = COMSOL_VTU(path_domain_3d)

In [6]:

# destination_mesh = domain_3d.mesh
# data_folder = Path(root / VERSION / DATA_TYPE)
# assert data_folder.exists(), f"Data folder {data_folder} does not exist."
# export_folder =  data_folder.parent / "Truncated"
# export_folder.mkdir(exist_ok=True)
# assert export_folder.exists(), f"Export folder {export_folder} does not exist."
# vtu_files = sorted([path for path in data_folder.iterdir() if path.suffix == ".vtu"])
# for idx, vtu_file in tqdm(enumerate(vtu_files), total=len(vtu_files), desc="Reading COMSOL files"):
#     temp_data = COMSOL_VTU(vtu_file)
#     temp_data = delete_pyvista_fields(temp_data, ["Temperature"])
#     mapped = map_on_control_mesh(temp_data.mesh, destination_mesh)
#     mapped.save(export_folder / vtu_file.name)

Reading COMSOL files: 100%|██████████| 20/20 [00:07<00:00,  2.56it/s]


In [7]:
COMSOL_VTU(export_folder / vtu_file.name)

COMSOL_VTU(vtu_path=PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Truncated/Test_019.vtu'), optional_vtu_paths=None, name='')

In [8]:
mapped.points.shape

(20113, 3)

In [9]:
destination_mesh.points.shape

(20113, 3)

In [ ]:
bounds = domain_3d.mesh.GetBounds()  # (xmin, xmax, ymin, ymax, zmin, zmax)

# Grid resolution
nx, ny, nz = 100, 100, 100  # You can increase this for more resolution

dx, dy, dz = 100, 100, 100  # You can increase this for more resolution
# Compute the number of points needed
nx = int((bounds[1] - bounds[0]) / dx) + 1
ny = int((bounds[3] - bounds[2]) / dy) + 1
nz = int((bounds[5] - bounds[4]) / dz) + 1


image = vtk.vtkImageData()
image.SetDimensions(nx, ny, nz)
image.SetOrigin(bounds[0], bounds[2], bounds[4])
image.SetSpacing(dx, dy, dz)
# image.SetSpacing(
#     (bounds[1] - bounds[0]) / (nx - 1),
#     (bounds[3] - bounds[2]) / (ny - 1),
#     (bounds[5] - bounds[4]) / (nz - 1),
# )

interpolated_vtk = map_on_control_mesh(domain_3d.mesh, image)
print(bounds)
print(f"{(bounds[1] - bounds[0]) / (nx)}" )
# pv_data = pv.wrap(image).save("interpolated_output.vti")

In [ ]:
path_domain_parameters = root /  PARAMETER_SPACE / "Exports" / f"{path_domain_3d.stem}_parameters.csv"
assert path_domain_parameters.exists()
df = pd.read_csv(path_domain_parameters, index_col = 0)
ureg = pint.UnitRegistry()
df['quantity_pint'] = df[df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))

In [27]:
df

,quantity,quantity_pint
aspect_ratio_dip,1.0 dimensionless,1.0 dimensionless
aspect_ratio_strike,1.25 dimensionless,1.25 dimensionless
beta,0.00059 / kelvin,0.00059 / kelvin
dip,1.0471975511965976 radian,1.0471975511965976 radian
faul_k_long,1.349373702338712e-15 meter ** 2,1.349373702338712e-15 meter ** 2
faul_k_trans,1.359969119986057e-18 meter ** 2,1.359969119986057e-18 meter ** 2
fault_Cp,800.0 meter ** 2 / kelvin / second ** 2,800.0 meter ** 2 / kelvin / second ** 2
fault_k_long,1.5e-13 meter ** 2,1.5e-13 meter ** 2
fault_k_trans,1.5e-13 meter ** 2,1.5e-13 meter ** 2
fault_lambda,2.0 kilogram * meter / kelvin / second ** 3,2.0 kilogram * meter / kelvin / second ** 3


In [28]:
def create_smooth_bounds(mesh: pv.DataSet, val_old : float, val_new: float, axis: str = 'z') -> pv.DataSet:
    map_axis_dict = {'x': 0, 'y': 1, 'z': 2}
    axis = map_axis_dict[axis]
    mesh.points[:, axis] = np.where(mesh.points[:, axis] == val_old, val_new, mesh.points[:, axis])
    return mesh

print(domain_3d.mesh.bounds)
# while domain_3d.mesh.bounds[-1] > 0.:
#     domain_3d.mesh = create_smooth_bounds(domain_3d.mesh, domain_3d.mesh.bounds[-1], 0. , axis='z')
# print(domain_3d.mesh.bounds)

(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)


In [29]:
domain_3d.info()

self.vtu_path=PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Test/Test_001.vtu')
41 timesteps from 0.000e+00 s to 4.000e+13 s
self.mesh.bounds=(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
Availabe fields in vtu dataset:
	 1: Temperature


In [30]:
dip    = df.loc["dip", "quantity_pint"].to("deg")
strike = df.loc["strike", "quantity_pint"].to("deg")
print(dip)
print(strike)
print(np.round(dip.magnitude))
print(np.round(strike.magnitude))

59.99999999999999 degree
90.0 degree
60.0
90.0


### Original Data

In [31]:
field_name = domain_3d.format_field("Temperature", -1)
normal_vector = calculate_normal(np.round(dip.magnitude), np.round(strike.magnitude))
clip_original = domain_3d.mesh.clip(normal = -normal_vector, origin=domain_3d.mesh.center)
# clip_original.plot(scalars = field_name, cmap=my_cmap)


### Control Mesh

In [32]:
bounds = domain_3d.mesh.bounds
xrng = np.arange(bounds[0], bounds[1] + 1e-6, 200)
yrng = np.arange(bounds[2], bounds[3] + 1e-6, 200)
zrng = np.arange(bounds[4], bounds[5] + 1e-6, 100)
grid = pv.RectilinearGrid(xrng, yrng, zrng)
grid.plot(show_edges=True)
print(grid.bounds)
print(domain_3d.mesh.bounds)

Widget(value='<iframe src="http://localhost:58760/index.html?ui=P_0x17f8acda0_0&reconnect=auto" class="pyvista…

(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)


### Add plane

In [33]:
control_mesh = grid  #pv.merge([grid, clipped_plane])
control_mesh.clip(normal = -normal_vector, origin=domain_3d.mesh.center).plot()
control_mesh_cell = control_mesh.copy()


Widget(value='<iframe src="http://localhost:58760/index.html?ui=P_0x30379f830_1&reconnect=auto" class="pyvista…

### pv.interpolate()

vtkGaussianKernel is an interpolation kernel that simply returns the weights for all points found in the sphere defined by radius R. The weights are computed as: $e^{-(\frac{s*r}{R})^2}$ where r is the distance from the point to be interpolated to a neighboring point within R. The sharpness s simply affects the rate of fall off of the Gaussian. (A more general Gaussian kernel is available from vtkEllipsoidalGaussianKernel.)

https://vtk.org/doc/nightly/html/classvtkGaussianKernel.html

Examples
https://examples.vtk.org/site/Cxx/Meshes/InterpolateFieldDataDemo/

In [34]:
result              = control_mesh.interpolate(domain_3d.mesh, radius=120, sharpness=1, strategy="mask_points")
clip_interpolate = result.clip(normal = -normal_vector, origin=domain_3d.mesh.center)

# Visualize
# clip_interpolate.plot(scalars=field_name, cmap=my_cmap, show_edges=False)
# result.plot(scalars=field_name, cmap="coolwarm", show_edges=True)

### pv.Sample()

In [35]:
cell_mesh = domain_3d.mesh.point_data_to_cell_data(pass_point_data=False)
# control_mesh_cell.plot()
# Visualize
# clip.plot(scalars=field_name, cmap=my_cmap, show_edges=False)
# result.plot(scalars=field_name, cmap="coolwarm", show_edges=True)

In [36]:
result_cell              = control_mesh_cell.sample(domain_3d.mesh)
clip_sample = result_cell.clip(normal = -normal_vector, origin=domain_3d.mesh.center)

### vtkProbeFilter

In [39]:
bounds = domain_3d.mesh.GetBounds()  # (xmin, xmax, ymin, ymax, zmin, zmax)

# Grid resolution
nx, ny, nz = 100, 100, 100  # You can increase this for more resolution

dx, dy, dz = 100, 100, 100  # You can increase this for more resolution
# Compute the number of points needed
nx = int((bounds[1] - bounds[0]) / dx) + 1
ny = int((bounds[3] - bounds[2]) / dy) + 1
nz = int((bounds[5] - bounds[4]) / dz) + 1


image = vtk.vtkImageData()
image.SetDimensions(nx, ny, nz)
image.SetOrigin(bounds[0], bounds[2], bounds[4])
image.SetSpacing(dx, dy, dz)
# image.SetSpacing(
#     (bounds[1] - bounds[0]) / (nx - 1),
#     (bounds[3] - bounds[2]) / (ny - 1),
#     (bounds[5] - bounds[4]) / (nz - 1),
# )

interpolated_vtk = map_on_control_mesh(domain_3d.mesh, image)
print(bounds)
print(f"{(bounds[1] - bounds[0]) / (nx)}" )
# pv_data = pv.wrap(image).save("interpolated_output.vti")
clip_vtk = interpolated_vtk.clip(normal = -normal_vector, origin=domain_3d.mesh.center)

(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
97.5609756097561


## Plot

In [40]:

# Loop through the samples and plot
clips = [clip_original, clip_interpolate, clip_sample, clip_vtk]
labels = ["Original", "pv.Interpolate()", "pv.Sample()", "vtkProbeFilter"]
plotter = pv.Plotter(shape=(1, len(clips)), window_size=(1200, 500))
for i, (clip, label) in enumerate(zip(clips, labels)):
    plotter.subplot(0, i)
    plotter.add_mesh(clip, scalars = field_name,
                     cmap = my_cmap,
                    scalar_bar_args={'title': f'{field_name} ({i})',
                    'label_font_size': 10,
                    'title_font_size': 8,}
                    )
    

    plotter.add_text(label)

plotter.show()

Widget(value='<iframe src="http://localhost:58760/index.html?ui=P_0x3383982c0_3&reconnect=auto" class="pyvista…